In [1]:
import pandas as pd
import numpy as np
import statistics
import datetime
import re

In [21]:
raw = pd.read_csv("data.csv")
# set(raw.columns)

/Users/eunicheng/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (14,20,28,31,32,56,68,69,72,73,76,77,78,80,81,82,83,84,86,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
#刪掉無資訊、明顯無相關的欄位
saved_columns = [
 'TypeIdName', #('其他', '犬', '貓')
 'AcceptDate', #被領養日期？
 'DistrictTeamName',
 'SexName',#{'none', '公', '未知', '母'}
 'ReasonName',#{nan, '不擬續養', '依法沒入', '其他', '動物救援', '拾獲送交', '收容所轉入', '政府捕捉'}
 'VarietyName',#種類，阿富汗、馬爾濟斯...71種，其中兩個是數字268,269
 'VarietyRemark',#更細的種類，沒有救NaN
 'BuildName',#體型名稱{'1', '中', '大', '小'}
 'Bodyweight',#超亂，setlen=12135, nan, NaN, 13~15, 2公斤, 4....
 'Age2Name',#{'1至3月齡', '1至3歲', '3至6月齡', '3至7歲', '6月齡至1歲', '7歲以上', nan, 'none', '未離乳'}
 'CoatName',#毛色，28種，包括'none'
 'MouthCoat',#亂，nan,'string', '鼻上方有一咖啡色胎記', '黑嘴','黑嘴    '
 'HeadCoat',# 亂，set多個nan???
 'ChestCoat',#亂，nan, 'string'...
 'FootCoat',#亂，nan, 'string'...
 'EarCoat',# 亂，set多個nan???
 'BackCoat',# 亂，set多個nan???
 'TailCoat',#亂，nan,'string',
 'CollarName',#項圈顏色{'none', '其他', '棕色', '無', '白色', '粉紅色', '紅色', '花色', '藍色', '銀色', '黃色', '黑色'}
#  'CollarRemark',#項圈特徵
 'ThoracodorsalName',#胸甲(衣服)顏色{'none', '其他', '棕色', '無', '白色', '紅色', '花色', '藍色', '黃色', '黑色'}
#  'ThoracodorsalRemark',#多，胸甲(衣服)特徵
 'AdoptionName',#{'待認領', '暫時不適合', '適合認養'}
 'NotAdoptionName',#{'none', '健康不適', '動保案件', '年紀不適', '行為不適'}
 '_Situation2Name',#{'none','人道處理其他法令授權','人道處理第12條第1項第3款','人道處理第12條第1項第5款', '人道處理第12條第1項第7款', '其他','原地飭回','收容所轉出','生理耗弱死亡','疾病死亡','認領','認養','逃脫'}
 'SituationRemark',#認養狀態備註
 'StatusName',#{'sNull', '其他', '原地飭回', '安樂死', '生理耗弱死亡', '疾病死亡', '認領養', '逃跑'}
#  'Address',
 'IsSterilizationName',#{'是', '未確定'}
#  'SterilizationDate',
 'EarNoteName',#{'none', '右剪耳', '左剪耳'}
#  'Note',#超雜
 'LengthCoatName',#{'none', '中長毛', '未修剪過', '短毛', '長毛'}
 'HealthStatusName',#{'none', '其它', '寄生蟲', '正常', '異常', '疫苗'}
#  'HealthStatusRemark',#超多受傷原因
 'Behavior',#幾種數字，但還不知道數字代表的意思
#  'BehaviorRemark',#超多行為描述
 'InImage',
 'ImageNameStr',
 'FrontImageName',
#  'VaccineDate', #沒填示沒打過疫苗？
#  'ShelterName',
#  'VillageID',
#  'VillageName',
 'FinalDate'#領養日期
]
raw = raw[saved_columns]

In [18]:
# raw.columns
rm_col = [i for i in raw.columns if i not in saved_columns]
print(', '.join(rm_col))

KeyNo, Name, TypeId, AcceptNum, DistrictTeam, Sex, Reason, ReasonType, ReasonTypeName, ReasonRemark, Variety, Build, Age, AgeName, Age2, Coat, HairType, Collar, CollarRemark, Thoracodorsal, ThoracodorsalRemark, Adoption, NotAdoption, Situation, SituationName, Situation2, ChipNum, DoglicenseNum, Remark, CreateTime, IsDelete, EscapeDate, EscapeRemark, Status, Address, IsSterilization, SterilizationDate, IsTNR, EarNote, DoctorId, OpenAdoptionTime, ShapeRemark, Story, IsIsolation, IsolationDate, IsolationReason, IsolationRemark, IsQuarantine, QuarantineDate, QuarantineReason, QuarantineRemark, VolunteerAlbum, Note, Resettlement, ChildreAnlong, AnimalAnlong, Phone, Email, AndroidCount, MovieUrl, IsChoice, isInto, SyncTime, CreateUser, UpdateUser, UpdateTime, LengthCoat, DoglicenseDate, DoglicensNoUseId, CageTypeName, CageName, TypeRemark, HealthStatus, HealthStatusRemark, BehaviorRemark, CageIdStr, CageTypeIdStr, MedicineName, VaccineDate, whVariety, WhInto, SearchKeyword, Unit, ShelterName

In [20]:
#dropna columns
dropna_col = ['FinalDate','ReasonName']
# replacenan_col = ['DistrictTeamName', 'VarietyRemark','Bodyweight',
#                   'MouthCoat','HeadCoat','ChestCoat','FootCoat','EarCoat',
#                   'BackCoat','TailCoat','HealthStatusRemark','BehaviorRemark','CollarRemark',
#                   'ThoracodorsalRemark','SituationRemark','Behavior','VillageName']
print("Before rm na:", raw.shape)

for d_col in dropna_col:
    raw=raw[raw[d_col].notna()]
    print(d_col, raw.shape)
raw.fillna('無', inplace=True)
raw.replace('none', '無', inplace=True)
raw['StatusName'].replace('sNull', '無', inplace=True)
print("Aefore rm na:", raw.shape)

Before rm na: (115267, 34)
FinalDate (115267, 34)
ReasonName (115261, 34)
Aefore rm na: (115261, 34)


In [21]:
#[Staydays]
raw['AcceptDate'] = [datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S") for x in raw['AcceptDate']] 
raw['FinalDate'] = [datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S") for x in raw['FinalDate']] 
raw["StayDays"] = raw["FinalDate"]-raw['AcceptDate']
raw['StayDays'] = [x.days for x in raw['StayDays']] 
#[Staydays] 清掉未被領養的資料
raw = raw[raw['StayDays']>=0]
raw.drop(columns=['AcceptDate', 'FinalDate'], inplace=True)

#[SexName]clean
raw["SexName"].replace({"none": "未知"}, inplace=True)
#VarietyName=>268,269清掉？

#[BuildName]clean
raw = raw[raw["BuildName"] != '1']
#[VarietyRemark]clean
tmp = [False if x in ['0','MIX','mix','string'] else True for x in raw['VarietyRemark']]
raw = raw[tmp]

print("After cleaning dirty data:", raw.shape)

After cleaning dirty data: (107115, 33)


In [22]:
#[Bodyweight]clean
tmp = [False if x in ['strin','東22','１',''] else True for x in raw['Bodyweight']]
raw = raw[tmp]
raw['Bodyweight'] = [str(x).replace('K', '').replace('G', '').replace('k', '').replace('g', '').replace('<', '').replace('-', '~').replace('ｋｇ', '').replace('公斤', '').replace('以上', '').replace('以下', '').replace('體重', '').replace('胖', '20').strip(". ") for x in raw['Bodyweight']]
tmp = []
for x in raw['Bodyweight']:
        if x == "無" or x == '':
            tmp.append("無")
            continue
        if x.find("~") != -1:
            tmp.append(statistics.mean([float(y) for y in x.split("~")]))
        else:
            tmp.append(float(x))
raw['Bodyweight'] = tmp
#[Bodyweight]remove outlier
tmp = np.array([x for x in tmp if x != "無"])
Q1= np.mean(tmp)-2*np.std(tmp)
Q3= np.mean(tmp)+2*np.std(tmp)
raw = raw[[True if x == "無" or (x >= Q1 and x <= Q3) else False for x in raw['Bodyweight']]]
#[Bodyweight]convert to category(0~5, 5~10, 10~15...20以上)
tmp = []
for x in raw['Bodyweight']:
    if x == "無":
        tmp.append("無")
    elif x > 0 and x <= 5:
        tmp.append("0~5")
    elif x > 5 and x <= 10:
        tmp.append("5~10")
    elif x > 10 and x <= 15:
        tmp.append("10~15")
    elif x > 15 and x <= 20:
        tmp.append("15~20")
    else:
        tmp.append("20以上")
raw['Bodyweight'] = tmp

print("After cleaning dirty data:", raw.shape)

After cleaning dirty data: (107088, 33)


# Coat 如何切分特徵，同時有多個特徵？

### 橘貓很受歡迎？？？

In [23]:
#ChestCoat
raw['ChestCoat'] = ["白毛" if x.find("白")!=-1 else x for x in raw['ChestCoat']] #496 objs
raw['ChestCoat'] = ["黃毛" if x.find("黃")!=-1 else x for x in raw['ChestCoat']] #7 objs
raw['ChestCoat'] = ["棕黑毛" if re.search('咖啡|黑|棕', x) else x for x in raw['ChestCoat']] #9 objs
raw['ChestCoat'] = [x if (x == "無" or re.search('白毛|黃毛|棕黑毛', x)) else "其他" for x in raw['ChestCoat']]
print("ChestCoat:")
for i in set(raw['ChestCoat']):
    print(i, sum([True if re.search(i, x) else False for x in raw['ChestCoat']]))
print()

#MouthCoat
raw['MouthCoat'] = ["白" if x.find("白")!=-1 else x for x in raw['MouthCoat']] #117 objs
raw['MouthCoat'] = ["黃" if x.find("黃")!=-1 else x for x in raw['MouthCoat']] #9 objs
raw['MouthCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x) else x for x in raw['MouthCoat']] #242 objs
raw['MouthCoat'] = [x if (x == "無" or re.search('白|黃|棕灰黑', x)) else "其他" for x in raw['MouthCoat']]
print("MouthCoat:")
for i in set(raw['MouthCoat']):
    print(i, sum([True if re.search(i, x) else False for x in raw['MouthCoat']]))
print()

#HeadCoat
raw['HeadCoat'] = ["虎斑" if re.search('虎斑|虎班', x) else x for x in raw['HeadCoat']] #23 objs
raw['HeadCoat'] = ["黑白" if re.search('黑.*白', x) else x for x in raw['HeadCoat']] #4 objs
raw['HeadCoat'] = ["棕灰黑" if (re.search('咖啡|黑|棕|灰', x) and x != "黑白") else x for x in raw['HeadCoat']] #41 objs
raw['HeadCoat'] = ["受傷" if re.search('傷|浮腫|受傷|車禍|獨眼|白內障', x) else x for x in raw['HeadCoat']] #10 objs
raw['HeadCoat'] = ["四眼" if re.search('四目|四眼', x) else x for x in raw['HeadCoat']] #66 objs
raw['HeadCoat'] = ["白" if (x.find("白")!=-1 and x != "黑白") else x for x in raw['HeadCoat']] #18 objs
raw['HeadCoat'] = ["黃" if x.find("黃")!=-1 else x for x in raw['HeadCoat']] #29 objs
raw['HeadCoat'] = [x if (x == "無" or re.search('虎斑|黑白|棕灰黑|受傷|四眼|白|黃', x)) else "其他" for x in raw['HeadCoat']]
print("HeadCoat:")
for i in set(raw['HeadCoat']):
    print(i, sum([True if re.search(i, x) else False for x in raw['HeadCoat']]))
print()

#FootCoat
raw['FootCoat'] = ["受傷" if re.search('傷|截|不良於行|骨折|斷裂|缺|腫瘤|斷|脫臼|尺骨外露|跛行|三腳|後癱|癱瘓|畸形|短|無力', x) else x for x in raw['FootCoat']] #10 objs
raw['FootCoat'] = ["虎斑" if re.search('虎斑|虎班|斑紋', x) else x for x in raw['FootCoat']] #23 objs
raw['FootCoat'] = ["白" if x.find("白")!=-1 else x for x in raw['FootCoat']] #18 objs
raw['FootCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x) else x for x in raw['FootCoat']] #23 objs
raw['FootCoat'] = ["黃" if x.find("黃")!=-1 else x for x in raw['FootCoat']] #18 objs
raw['FootCoat'] = [x if (x == "無" or re.search('虎斑|白|棕灰黑|受傷|黃', x)) else "其他" for x in raw['FootCoat']]
print("FootCoat:")
for i in set(raw['FootCoat']):
    print(i, sum([True if re.search(i, x) else False for x in raw['FootCoat']]))
print()

#EarCoat
raw['EarCoat'] = ["剪/缺/折/立/垂耳" if re.search('垂|剪|折|缺|立|摺|聳', x) else x for x in raw['EarCoat']] #10 objs
raw['EarCoat'] = ["受傷" if re.search('左耳皺|皮膚炎|長瘤', x) else x for x in raw['EarCoat']] #23 objs
raw['EarCoat'] = ["虎斑" if re.search('虎斑|虎班|斑紋', x) else x for x in raw['EarCoat']] #23 objs
raw['EarCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x) else x for x in raw['EarCoat']] #23 objs
raw['EarCoat'] = ["黃" if x.find("黃")!=-1 else x for x in raw['EarCoat']] #18 objs
raw['EarCoat'] = [x if (x == "無" or re.search('虎斑|缺|棕灰黑|受傷|黃', x)) else "其他" for x in raw['EarCoat']]
print("EarCoat:")
for i in set(raw['EarCoat']):
    print(i, sum([True if re.search(i, x) else False for x in raw['EarCoat']]))
print()

#BackCoat
raw['BackCoat'] = ["受傷" if re.search('皮膚病|傷|突出物', x) else x for x in raw['BackCoat']]
raw['BackCoat'] = ["斑點" if re.search('虎斑|虎班|斑紋|斑', x) else x for x in raw['BackCoat']]
raw['BackCoat'] = ["白" if re.search('白', x) else x for x in raw['BackCoat']]
raw['BackCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x)  else x for x in raw['BackCoat']] 
raw['BackCoat'] = [x if (x == "無" or re.search('斑點|棕灰黑|受傷|白', x)) else "其他" for x in raw['BackCoat']]
print("BackCoat:")
for i in set(raw['BackCoat']):
    print(i, sum([True if re.search(i, x) else False for x in raw['BackCoat']]))
print()

#TailCoat
raw['TailCoat'] = ["有病" if re.search('皮膚病|肉瘤', x) else x for x in raw['TailCoat']]
raw['TailCoat'] = ["截尾" if re.search('截|剪尾|半短|折尾|斷|短|半尾', x) else x for x in raw['TailCoat']]
raw['TailCoat'] = ["無尾" if re.search('無尾', x) else x for x in raw['TailCoat']]
raw['TailCoat'] = ["斑點" if re.search('虎斑|虎班|斑紋|斑', x) else x for x in raw['TailCoat']]
raw['TailCoat'] = ["麒麟尾" if re.search('麒麟', x) else x for x in raw['TailCoat']]
raw['TailCoat'] = ["鐮刀尾" if re.search('鐮刀', x) else x for x in raw['TailCoat']]
raw['TailCoat'] = ["白" if re.search('白', x) else x for x in raw['TailCoat']]
raw['TailCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x)  else x for x in raw['TailCoat']] 
raw['TailCoat'] = [x if (x == "無" or re.search('斑點|棕灰黑|有病|白|截尾|無尾|麒麟尾|鐮刀尾', x)) else "其他" for x in raw['TailCoat']]
print("TailCoat:")
set(raw['TailCoat'])
for i in set(raw['TailCoat']):
    print(i, sum([True if re.search(i, x) else False for x in raw['TailCoat']]))
print()

ChestCoat:
無 106445
其他 172
黃毛 7
棕黑毛 7
白毛 457

MouthCoat:
黃 5
無 106579
白 109
其他 179
棕灰黑 216

HeadCoat:
黃 26
無 106731
白 16
其他 186
四眼 63
受傷 9
黑白 3
虎斑 19
棕灰黑 38

FootCoat:
黃 95
無 106330
白 319
其他 180
受傷 41
虎斑 78
棕灰黑 45

EarCoat:
剪/缺/折/立/垂耳 155
黃 15
無 106710
其他 170
受傷 7
虎斑 7
棕灰黑 24

BackCoat:
無 106836
白 16
其他 183
斑點 12
受傷 9
棕灰黑 32

TailCoat:
麒麟尾 52
截尾 58
無 106746
白 9
其他 183
無尾 5
斑點 7
棕灰黑 31
有病 2



In [24]:
#有無項圈
raw['CollarName'] = ["無項圈" if x == "無" else "有項圈" for x in raw['CollarName']]
#有無衣服
raw['ThoracodorsalName'] = ["無衣服" if x == "無" else "有衣服" for x in raw['ThoracodorsalName']]

# NotAdoptionName有標注的欄位都刪掉？

In [19]:
#每隻動物都被領養了嗎？？？因為有NotAdoptionName這個欄位
print(set(raw['NotAdoptionName']))
raw = raw[raw['NotAdoptionName']=='無']
raw.drop(columns=['NotAdoptionName'])
print("rm [NotAdoptionName], data shape", raw.shape)

{'健康不適', '年紀不適', '行為不適', '動保案件', 'none'}
rm [NotAdoptionName], data shape (0, 128)


# SituationRemark有標注的欄位都刪掉？

In [23]:
print(set(raw['SituationRemark']))
raw = raw[raw['SituationRemark']=='無']
raw.drop(columns=['SituationRemark'])
print("rm [SituationRemark], data shape", raw.shape)

{nan, '撤銷', '1/27中途認養', '10/16認養', '107/06/08飼主領回', '990001000048132', '代養人將犬隻直接交予原飼主,該犬隻,現寵登,登記在中和動物之家名下 ', '鐘文虎領回', '990000001994702', '8/11FPV(+)', '990001000052790', '990000002712361', '5/4TNR', '5/19死亡送回', '5/4自然死亡', '不吃，流濃稠口水，流鼻涕', '990000002710650', '原TNVR 剪右耳-交范S', '8/15由拾獲人帶出；10月初致電詢問犬隻尚未絕育，已提醒盡快帶去手術', '107/01/03', '12/1TNVR', '花990000002710661', '左前肢化膿壞死,左前胸腹側水腫', '2/14中途-呂', '骨科手術', '6/11死亡送回', '2020/01/30 民眾認養', '家人代理領回', '10/31TNVR', '7/10TNVR', '9/11中途認養', "107'11/29施打2合1疫苗", '犬小病毒感染', '990000001994908', '990000001405191', '990000002711243', 'TNVR', 'TNR(出沒地點:七堵區大同街)', '花園990000002711506', '8/10TNVR', '虛弱需醫療', '107/06/21認養', '放風時脫逃', '由樹林區楊櫂先生先於108年12月18日認養', 'CPV', '轉出八里動物之家 後續晶片協尋', '轉板橋動物之家', '990000002712371', '1080109頭份愛心小站送養', '990000002711378', '108/06/28認養', '990000001994806', '109/01/10民眾認養', '送奧斯卡鼎山櫥窗', '20200111由翁家玉小姐至本處認養', '7/3TNVR', '狗咬傷死亡', '花園990001000052595', '990000002711368', '4/25死亡送回', '7/3認養\\7/16寵登', '107/1/10陳浩文領養', '990000001994699', '990000002711

# IsSterilizationName'填的有{'是', '未確定'}，填是的有14834筆，SterilizationDate 有填的9961筆


### 要保留IsSterilizationName這個欄位就好嗎？

In [27]:
# print(sum([True if x!='0001-01-01T00:00:00' else False for x in raw['SterilizationDate']]))
# print(sum([True if x=='是' else False for x in raw['IsSterilizationName']]))
# set(raw['SterilizationDate'])

#  備註欄位直接刪掉？內容大致為三類（病因、外觀、各種阿里阿咂），其中病因佔多數

In [28]:
# print(sum([True if x=='無' else False for x in raw['Note']]))
# set(raw['Note'])

In [29]:
# set(raw['HealthStatusName'])
# print(sum([True if x=='無' else False for x in raw['HealthStatusName']]))

#  健康狀態備註欄位直接刪掉？
- 絕育(佔多數)(紮)
- 病因or缺陷
- 疫苗（CCV\CPV\FIV\五合一）
- 驅蟲藥
- 幼犬奶貓

In [30]:
# set(raw['HealthStatusRemark'])
# print(sum([True if x=='無' else False for x in raw['HealthStatusRemark']]))

# [Behavior]可以有多種category，切成01表

In [31]:
tmp = []
for a in raw['Behavior']:
    tmp += [x for x in a.split(", ")]
health_types = set(tmp)
for i in health_types:
    health_types_col = []
    for a in raw['Behavior']:
        if i in a:
            health_types_col.append(1)
        else:
            health_types_col.append(0)
    raw['Behavior'+i] = health_types_col
raw.drop(columns=['Behavior'], inplace=True)

In [32]:
file = open('clean1_description.txt', 'w')
file.write('# of columns:'+str(len(raw.columns)))
raw = raw.astype(str)
for col in raw.columns:
    if col in ['InImage','ImageNameStr','FrontImageName']:
        file.write(col+" : manymany")
        continue
    print(col+' : ', set(raw[col]))
    file.write(col+' : '+', '.join(list(set(raw[col])))+'\n')
#     print(i, sum([True if re.search(i, x) else False for x in raw['TailCoat']]))
file.close()
raw.to_csv('clean1.csv', index=False)

TypeIdName :  {'貓', '其他', '犬'}
DistrictTeamName :  {'潮州鎮', '田中鎮', '大肚區', '松山區', '新埔鎮', '卓蘭鎮', '關廟區', '高樹鄉', '國姓鄉', '望安鄉', '峨眉鄉', '新屋區', '左營區', '萬榮鄉', '和平區', '鳥松區', '彌陀區', '二林鎮', '卑南鄉', '延平鄉', '車城鄉', '湖西鄉', '金峰鄉', '汐止區', '梓官區', '宜蘭市', '萬巒鄉', '口湖鄉', '大雅區', '九如鄉', '社頭鄉', '梧棲區', '溪州鄉', '竹田鄉', '芬園鄉', '名間鄉', '元長鄉', '中山區', '東港鎮', '竹崎鄉', '枋山鄉', '三義鄉', '大村鄉', '南區', '安定區', '羅東鎮', '旗山區', '泰武鄉', '安南區', '安平區', '萬里區', '豐原區', '大社區', '新莊區', '美濃區', '寶山鄉', '福興鄉', '枋寮鄉', '溪湖鎮', '獅子鄉', '林口區', '公館鄉', '銅鑼鄉', '三峽區', '光復鄉', '新營區', '莒光鄉', '東石鄉', '石門區', '麻豆區', '霧峰區', '大內區', '竹塘鄉', '龍井區', '左鎮區', '三灣鄉', '秀水鄉', '橋頭區', '大林鎮', '民雄鄉', '義竹鄉', '北竿鄉', '外埔區', '大武鄉', '尖石鄉', '永安區', '新園鄉', '山上區', '平溪區', '龜山區', '復興區', '太保市', '土庫鎮', '造橋鄉', '六龜區', '仁武區', '內湖區', '金湖鎮', '南化區', '鶯歌區', '坪林區', '七股區', '柳營區', '無', '芎林鄉', '蘇澳鎮', '龍潭區', '大同區', '瑪家鄉', '虎尾鎮', '草屯鎮', '沙鹿區', '燕巢區', '新社區', '萬華區', '北埔鄉', '白沙鄉', '中寮鄉', '吉安鄉', '竹東鎮', '東區', '中正區', '淡水區', '內埔鄉', '五結鄉', '下營區', '新埤鄉', '清水區', '湖內區', '七堵區', '員山鄉', '善化區', '苓雅區', '大樹區', '花壇鄉', '鳳山區', 

Behavior85 :  {'0', '1'}
Behavior88 :  {'0', '1'}
